# Spotify API Feature Extraction

* Spotify Mlllion Playlist Dataset Challenge [Homepage](https://www.aicrowd.com/challenges/spotify-million-playlist-dataset-challenge)
* [Spotify Web API docs](https://developer.spotify.com/documentation/web-api/reference/#/)

**Community Examples**
* [Extracting song lists](https://github.com/tojhe/recsys-spotify/blob/master/processing/songlist_extraction.py)
* [construct audio features with Spotify API](https://github.com/tojhe/recsys-spotify/blob/master/processing/audio_features_construction.py)
* [Using Spotify API](https://towardsdatascience.com/extracting-song-data-from-the-spotify-api-using-python-b1e79388d50)

### pip & package 

In [ ]:
PROJECT_ID = 'jtotten-project' #update
# PROJECT_ID = 'matching-engine-playlist'  # <--- TODO: CHANGE THIS
LOCATION = 'us-central1' 
!gcloud config set project {PROJECT_ID}

Updated property [core/project].


In [ ]:
import sys
if 'google.colab' in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

In [ ]:
if 'google.colab' in sys.modules:
    USER_FLAG = ''
else:
    USER_FLAG = '--user'

In [ ]:
# spotify
!pip install spotipy

! pip3 install -U google-cloud-storage $USER_FLAG
! pip3 install $USER kfp google-cloud-pipeline-components --upgrade
# !git clone https://github.com/kubeflow/pipelines.git
# !pip install pipelines/components/google-cloud/.

# stable Vertex SDK
!pip install google-cloud-aiplatform

!pip install gcsfs
!pip install fsspec

# Download and install latest (preview) version of Vertex SDK
# !pip install -U git+https://github.com/ivanmkc/python-aiplatform.git@imkc--matching-engine

# OSS Scann for testing (OSS)
# !pip install scann

# Automatically restart kernel after installs
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

     |████████████████████████████████| 63 kB 1.4 MB/s 
     |████████████████████████████████| 138 kB 19.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


     |████████████████████████████████| 107 kB 10.4 MB/s 
     |████████████████████████████████| 114 kB 50.2 MB/s 
     |████████████████████████████████| 76 kB 4.7 MB/s 
  Attempting uninstall: google-api-core
    Found existing installation: google-api-core 1.26.3
    Uninstalling google-api-core-1.26.3:
      Successfully uninstalled google-api-core-1.26.3
  Attempting uninstall: google-resumable-media
    Found existing installation: google-resumable-media 0.4.1
    Uninstalling google-resumable-media-0.4.1:
      Successfully uninstalled google-resumable-media-0.4.1
  Attempting uninstall: google-cloud-core
    Found existing installation: google-cloud-core 1.0.3
    Uninstalling google-cloud-core-1.0.3:
      Successfully uninstalled google-cloud-core-1.0.3
  Attempting uninstall: google-cloud-storage
    Found existing installation: google-cloud-storage 1.18.1
    Uninstalling google-cloud-storage-1.18.1:
      Successfully uninstalled google-cloud-storage-1.18.1
ERROR: pip's d

     |████████████████████████████████| 301 kB 8.5 MB/s 
     |████████████████████████████████| 375 kB 47.1 MB/s 
     |████████████████████████████████| 636 kB 49.6 MB/s 
     |████████████████████████████████| 106 kB 55.9 MB/s 
     |████████████████████████████████| 1.6 MB 57.5 MB/s 
     |████████████████████████████████| 54 kB 2.5 MB/s 
     |████████████████████████████████| 54 kB 2.3 MB/s 
     |████████████████████████████████| 56 kB 4.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 87 kB 6.1 MB/s 
     |████████████████████████████████| 10.9 MB 47.5 MB/s 
     |████████████████████████████████| 54 kB 2.4 MB/s 
     |████████████████████████████████| 343 kB 68.9 MB/s 
     |████████████████████████████████| 1.8 MB 43.7 MB/s 
     |████████████████████████████████| 46 kB 3.5 MB/s 
     |████████████████████████████████| 206 kB 49.3 MB/s 
     |██████████

     |████████████████████████████████| 1.1 MB 12.2 MB/s 
     |████████████████████████████████| 136 kB 57.4 MB/s 
     |████████████████████████████████| 271 kB 45.3 MB/s 
     |████████████████████████████████| 94 kB 2.6 MB/s 
     |████████████████████████████████| 144 kB 59.4 MB/s 


{'restart': True, 'status': 'ok'}

In [ ]:
! python3 -c "import kfp; print('KFP SDK version: {}'.format(kfp.__version__))"
! python3 -c "import google_cloud_pipeline_components; print('google_cloud_pipeline_components version: {}'.format(google_cloud_pipeline_components.__version__))"
! python3 -c "import google.cloud.aiplatform; print('aiplatform SDK version: {}'.format(google.cloud.aiplatform.__version__))"

KFP SDK version: 1.8.12
google_cloud_pipeline_components version: 1.0.2
INFO:numexpr.utils:NumExpr defaulting to 2 threads.
aiplatform SDK version: 1.11.0


In [ ]:
LOCATION = 'us-central1'
PROJECT_ID = 'jtotten-project'
# PROJECT_ID = 'matching-engine-playlist'

!gcloud config set project {PROJECT_ID}

import os
os.environ['GOOGLE_CLOUD_PROJECT']=PROJECT_ID

# Required Pipeline Parameters
PIPE_USER = 'jtott'  #  {type: 'string'} <--- TODO: CHANGE THIS

BUCKET_NAME = 'matching-engine-content'
# BUCKET_NAME = 'spotify-million-playlists'

GS_PIPELINE_ROOT_PATH = 'gs://{}/pipeline_root/{}'.format(BUCKET_NAME, PIPE_USER)

from google.colab import auth as colab_auth
colab_auth.authenticate_user()

Are you sure you wish to set property [core/project] to jtotten-project?

Do you want to continue (Y/n)?  Y

Updated property [core/project].


In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import re
from tqdm import tqdm

import pandas as pd
import numpy as np
import json
from io import BytesIO

import time

import gcsfs

# GCP
from google.cloud import aiplatform
from google.cloud import storage

# Pipelines
from typing import Any, Callable, Dict, NamedTuple, Optional, List
from google_cloud_pipeline_components import aiplatform as gcc_aip
from google_cloud_pipeline_components.types import artifact_types

# Kubeflow SDK
# TODO: fix these
from kfp.v2 import dsl
import kfp
import kfp.v2.dsl
from kfp.v2.google import client as pipelines_client
from kfp.v2.dsl import (Artifact, Dataset, Input, InputPath, Model, Output,
                        OutputPath, component)


import tensorflow as tf

pd.set_option('display.max_columns', 100)

### clients & credentials

GCP clients

In [ ]:
# Setup clients

storage_client = storage.Client(project=PROJECT_ID
)

pipeline_client = pipelines_client.AIPlatformClient(
  project_id=PROJECT_ID,
  region=LOCATION,
)

aiplatform.init(
    project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_NAME
)

/usr/local/lib/python3.7/dist-packages/kfp/v2/google/client/client.py:173: FutureWarning: AIPlatformClient will be deprecated in v2.0.0. Please use PipelineJob https://googleapis.dev/python/aiplatform/latest/_modules/google/cloud/aiplatform/pipeline_jobs.html in Vertex SDK. Install the SDK using "pip install google-cloud-aiplatform"
  category=FutureWarning,


Pipeline setup

In [ ]:
# Pipeline Stuff
import os
PIPELINES = {}

PIPELINES_FILEPATH = f'gs://{BUCKET_NAME}/pipelines/pipelines.json' # <--- TODO: CHANGE THIS; can be blank json file
print("PIPELINES_FILEPATH:", PIPELINES_FILEPATH)

if os.path.isfile(PIPELINES_FILEPATH):
    with open(PIPELINES_FILEPATH) as f:
        PIPELINES = json.load(f)
else:
    PIPELINES = {}

def save_pipelines():
    with open(PIPELINES_FILEPATH, 'w') as f:
        json.dump(PIPELINES, f)

PIPELINES_FILEPATH: gs://matching-engine-content/pipelines/pipelines.json


# Create Pipeline Components

### Union all the slices for single artist and audio feature table

In [ ]:
%%bigquery
CREATE OR REPLACE TABLE `jtotten-project.spotify_mpd.artists` as 
(select * from
`jtotten-project.spotify_mpd.artist_split_1`
UNION ALL
select * from
`jtotten-project.spotify_mpd.artist_split_2`
UNION ALL
select * from
`jtotten-project.spotify_mpd.artist_split_3`
UNION ALL
select * from
`jtotten-project.spotify_mpd.artist_split_4`
UNION ALL
select * from
`jtotten-project.spotify_mpd.artist_split_5`
UNION ALL
select * from
`jtotten-project.spotify_mpd.artist_split_6`
UNION ALL
select * from
`jtotten-project.spotify_mpd.artist_split_7`
UNION ALL
select * from
`jtotten-project.spotify_mpd.artist_split_8`
UNION ALL
select * from
`jtotten-project.spotify_mpd.artist_split_9`
UNION ALL
select * from
`jtotten-project.spotify_mpd.artist_split_10`
UNION ALL
select * from
`jtotten-project.spotify_mpd.artist_split_11`
)

Query complete after 0.01s: 100%|██████████| 3/3 [00:00<00:00, 559.29query/s]


""




```
# This is formatted as code
```

## Split tracks data

In [ ]:
@kfp.v2.dsl.component(
    base_image="python:3.9",
    packages_to_install=['fsspec',
                         'google-cloud-storage',
                         'gcsfs',
                        #  'spotipy',
                         'numpy','pandas','pyarrow',
                         'absl-py',],
)
def split_tracks_data(
    project: str,
    location: str,
    gcs_bucket: str,
    gcs_prefix_eda: str,
    all_tracks_filename: str,
) -> NamedTuple('Outputs', [('all_tracks_split_1_gcs_uri', str),
                            ('all_tracks_split_2_gcs_uri', str),
                            ('all_tracks_split_3_gcs_uri', str),
                            ('all_tracks_split_4_gcs_uri', str),
                            ('all_tracks_split_5_gcs_uri', str),
                            ('all_tracks_split_6_gcs_uri', str),
                            ('all_tracks_split_7_gcs_uri', str),
                            ('all_tracks_split_8_gcs_uri', str),
                            ('all_tracks_split_9_gcs_uri', str),
                            ('all_tracks_split_10_gcs_uri', str),
                            ('all_tracks_split_11_gcs_uri', str),
                            ]):
  
  import os
  # import spotipy
  # from spotipy.oauth2 import SpotifyClientCredentials
  import re
  import pandas as pd
  import json
  from io import BytesIO
  import time
  from google.cloud import storage
  import gcsfs
  import numpy as np
  from absl import logging

  logging.set_verbosity(logging.INFO)

  storage_client = storage.Client(
    project=project
  )

  df = pd.read_csv(f'gs://{gcs_bucket}/{gcs_prefix_eda}/{all_tracks_filename}')
  # print(f'Shape of {all_tracks_filename}: {df.shape}')
  logging.info(f'Shape of {all_tracks_filename}: {df.shape}')
  
  all_tracks_split_1, all_tracks_split_2, all_tracks_split_3, all_tracks_split_4, all_tracks_split_5, all_tracks_split_6, all_tracks_split_7, all_tracks_split_8, all_tracks_split_9, all_tracks_split_10, all_tracks_split_11 = np.array_split(df, 11)#turning up to 11

  # upload split to GCS
  bucket = storage_client.get_bucket(f'{gcs_bucket}')

  # ONE
  bucket.blob(f'{gcs_prefix_eda}/all_tracks_split_1.csv').upload_from_string(
      all_tracks_split_1.to_csv(index=False), 'text/csv')
  
  all_tracks_split_1_gcs_uri = f'gs://{gcs_bucket}/{gcs_prefix_eda}/all_tracks_split_1.csv'

  # TWO
  bucket.blob(f'{gcs_prefix_eda}/all_tracks_split_2.csv').upload_from_string(
      all_tracks_split_2.to_csv(index=False), 'text/csv')
  
  all_tracks_split_2_gcs_uri = f'gs://{gcs_bucket}/{gcs_prefix_eda}/all_tracks_split_2.csv'

  # THREE
  bucket.blob(f'{gcs_prefix_eda}/all_tracks_split_3.csv').upload_from_string(
      all_tracks_split_3.to_csv(index=False), 'text/csv')
  
  all_tracks_split_3_gcs_uri = f'gs://{gcs_bucket}/{gcs_prefix_eda}/all_tracks_split_3.csv'

  # FOUR
  bucket.blob(f'{gcs_prefix_eda}/all_tracks_split_4.csv').upload_from_string(
      all_tracks_split_4.to_csv(index=False), 'text/csv')
  
  all_tracks_split_4_gcs_uri = f'gs://{gcs_bucket}/{gcs_prefix_eda}/all_tracks_split_4.csv'

  logging.info(f'all_tracks_split_1_gcs_uri {all_tracks_split_1_gcs_uri}')
  logging.info(f'all_tracks_split_2_gcs_uri {all_tracks_split_2_gcs_uri}')
  logging.info(f'all_tracks_split_3_gcs_uri {all_tracks_split_3_gcs_uri}')
  logging.info(f'all_tracks_split_4_gcs_uri {all_tracks_split_4_gcs_uri}')

    # FIVE
  bucket.blob(f'{gcs_prefix_eda}/all_tracks_split_5.csv').upload_from_string(
      all_tracks_split_5.to_csv(index=False), 'text/csv')
  
  all_tracks_split_5_gcs_uri = f'gs://{gcs_bucket}/{gcs_prefix_eda}/all_tracks_split_5.csv'

  # SIX
  bucket.blob(f'{gcs_prefix_eda}/all_tracks_split_6.csv').upload_from_string(
      all_tracks_split_6.to_csv(index=False), 'text/csv')
  
  all_tracks_split_6_gcs_uri = f'gs://{gcs_bucket}/{gcs_prefix_eda}/all_tracks_split_6.csv'

  # SEVEN
  bucket.blob(f'{gcs_prefix_eda}/all_tracks_split_7.csv').upload_from_string(
      all_tracks_split_7.to_csv(index=False), 'text/csv')
  
  all_tracks_split_7_gcs_uri = f'gs://{gcs_bucket}/{gcs_prefix_eda}/all_tracks_split_7.csv'

  # EIGHT
  bucket.blob(f'{gcs_prefix_eda}/all_tracks_split_8.csv').upload_from_string(
      all_tracks_split_8.to_csv(index=False), 'text/csv')
  
  all_tracks_split_8_gcs_uri = f'gs://{gcs_bucket}/{gcs_prefix_eda}/all_tracks_split_8.csv'

  logging.info(f'all_tracks_split_5_gcs_uri {all_tracks_split_5_gcs_uri}')
  logging.info(f'all_tracks_split_6_gcs_uri {all_tracks_split_6_gcs_uri}')
  logging.info(f'all_tracks_split_7_gcs_uri {all_tracks_split_7_gcs_uri}')
  logging.info(f'all_tracks_split_8_gcs_uri {all_tracks_split_8_gcs_uri}')

  # NINE
  bucket.blob(f'{gcs_prefix_eda}/all_tracks_split_9.csv').upload_from_string(
      all_tracks_split_9.to_csv(index=False), 'text/csv')
  
  all_tracks_split_9_gcs_uri = f'gs://{gcs_bucket}/{gcs_prefix_eda}/all_tracks_split_9.csv'

  # TEN
  bucket.blob(f'{gcs_prefix_eda}/all_tracks_split_10.csv').upload_from_string(
      all_tracks_split_10.to_csv(index=False), 'text/csv')
  
  all_tracks_split_10_gcs_uri = f'gs://{gcs_bucket}/{gcs_prefix_eda}/all_tracks_split_10.csv'

  # ELEVEN
  bucket.blob(f'{gcs_prefix_eda}/all_tracks_split_11.csv').upload_from_string(
      all_tracks_split_11.to_csv(index=False), 'text/csv')
  
  all_tracks_split_11_gcs_uri = f'gs://{gcs_bucket}/{gcs_prefix_eda}/all_tracks_split_11.csv'


  logging.info(f'all_tracks_split_9_gcs_uri {all_tracks_split_9_gcs_uri}')
  logging.info(f'all_tracks_split_10_gcs_uri {all_tracks_split_10_gcs_uri}')
  logging.info(f'all_tracks_split_11_gcs_uri {all_tracks_split_11_gcs_uri}')


  return(
      f'{all_tracks_split_1_gcs_uri}',
      f'{all_tracks_split_2_gcs_uri}',
      f'{all_tracks_split_3_gcs_uri}',
      f'{all_tracks_split_4_gcs_uri}',
      f'{all_tracks_split_5_gcs_uri}',
      f'{all_tracks_split_6_gcs_uri}',
      f'{all_tracks_split_7_gcs_uri}',
      f'{all_tracks_split_8_gcs_uri}',
      f'{all_tracks_split_9_gcs_uri}',
      f'{all_tracks_split_10_gcs_uri}',
      f'{all_tracks_split_11_gcs_uri}',
  )

## Call Spotify Split

### Audio

In [ ]:
@kfp.v2.dsl.component(
    base_image="python:3.9",
    packages_to_install=['fsspec', 'google-cloud-bigquery',
                         'google-cloud-storage',
                         'gcsfs',
                         'spotipy','requests','db-dtypes',
                         'numpy','pandas','pyarrow','absl-py', 'pandas-gbq==0.17.4'])
def call_spotify_api_split_audio(
    project: str,
    location: str,
    all_tracks_split_1_gcs_uri: str,
    gcs_bucket: str,
    gcs_prefix_eda: str,
    client_id: str,
    client_secret: str,
    sleep_param: int,
    split_id: int,
) -> NamedTuple('Outputs', [('track_audio_feat_split_1_gcs_uri', str),]):
  print(f'pip install complete')
  import os
  import spotipy
  from spotipy.oauth2 import SpotifyClientCredentials
  import re
  import warnings
  warnings.simplefilter(action='ignore', category=FutureWarning)
  import pandas as pd
  import json
  from io import BytesIO
  import time
  from google.cloud import storage
  import gcsfs
  import numpy as np
  from requests.exceptions import ReadTimeout, HTTPError, ConnectionError, RequestException
  from absl import logging
  from google.cloud import bigquery
  import pandas_gbq

  # print(f'package import complete')

  logging.set_verbosity(logging.INFO)
  logging.info(f'package import complete')

  storage_client = storage.Client(
    project=project
  )
  split=f'split_{split_id}'

  logging.info(f'Extracting audio features and recommendations for {split} \n')


  logging.info(f'spotipy auth complete')
  def spot_audio_features(uri, client_id, client_secret):
    
    # Authenticate
    client_credentials_manager = SpotifyClientCredentials(
        client_id=client_id, 
        client_secret=client_secret
    )
    sp = spotipy.Spotify(
        client_credentials_manager = client_credentials_manager, 
        requests_timeout=10, 
        retries=10 )
    ############################################################################
    # Create Track Audio Features DF for Split
    ############################################################################

    #Audio features
    uri = [u.replace('"', '') for u in uri] #fix the quotes 
    a_feats = sp.audio_features(uri)
    features = pd.json_normalize(a_feats, ).to_dict('list')
    if features is None:
      features = {}
    #Artist of the track, for genres and popularity
    popularity = []
    #tracks API call
    tracks = sp.tracks(uri)
    # if tracks:
    for track in tracks['tracks']:
      if track is not None:
        popularity.append(track['popularity'])
      else:
        popularity.append(-1)

    audio_df = pd.DataFrame(features)
    audio_df['popularity'] = popularity
    audio_df['track_uri'] = uri
    return audio_df

  bq_client = bigquery.Client(
      project=project, location=location
  )

  audio_featureDF = pd.DataFrame()
 
  query = f"""select count(1) as count from `jtotten-project.spotify_mpd.unique_tracks` a where mod(a.duration_ms, 11) = {split_id}-1 
  and a.track_uri NOT IN (SELECT track_uri FROM `jtotten-project.spotify_mpd.track_audio_{split_id}`)""" #excluding what we already loaded
  uri_list_length_df = bq_client.query(query).result().to_dataframe()
  uri_list_length = uri_list_length_df['count'][0]
  
  count = 1
  uri_batch = []
  # handling bad track/artist_uris

  #refactor
  schema = [{'name':'danceability', 'type': 'FLOAT'},	
            {'name':'energy', 'type': 'FLOAT'},	
            {'name':'key', 'type': 'FLOAT'},	
            {'name':'loudness', 'type': 'FLOAT'},	
            {'name':'mode', 'type': 'FLOAT'	},	
            {'name':'speechiness', 'type': 'FLOAT'	},	
            {'name':'acousticness', 'type': 'FLOAT'	},	
            {'name':'instrumentalness', 'type': 'FLOAT'	},	
            {'name':'liveness', 'type': 'FLOAT'	},	
            {'name':'valence', 'type': 'FLOAT'	},	
            {'name':'followers', 'type': 'FLOAT'	},	
            {'name':'tempo', 'type': 'FLOAT'},
            {'name':'type', 'type': 'STRING'},
            {'name':'id', 'type': 'STRING'},
            {'name':'uri', 'type': 'STRING'},
            {'name':'track_href', 'type': 'STRING'},
            {'name':'analysis_url', 'type': 'STRING'},
            {'name':'duration_ms_y', 'type': 'FLOAT'},
            {'name':'time_signature', 'type': 'FLOAT'},
            {'name':'popularity', 'type': 'INT64'},
            {'name':'track_uri', 'type': 'STRING'},
  ]
  query = f"""select track_uri from `jtotten-project.spotify_mpd.unique_tracks` a where mod(a.duration_ms, 11) = {split_id}-1 
  and a.track_uri NOT IN (SELECT track_uri FROM `jtotten-project.spotify_mpd.track_audio_{split_id}`)""" #excluding what we already loaded

  tracks = bq_client.query(query).result().to_dataframe()
  track_list = tracks.track_uri.to_list()
  for uri in track_list:
    if count % 50 == 0 or uri_list_length == count: #grab a batch of 50 songs
      uri_batch.append(uri)
      ### Try catch block for function
      try:
        audio_featureDF = spot_audio_features(uri_batch, client_id, client_secret)
        
      except ReadTimeout:
        logging.info("'Spotify timed out... trying again...'")
        audio_featureDF = spot_audio_features(uri_batch, client_id, client_secret)
      except HTTPError as err: #JW ADDED
        logging.info(f"HTTP Error: {err}")
      except spotipy.exceptions.SpotifyException as spotify_error: #jw_added
        logging.info(f"Spotify error: {spotify_error}")
      try:
        audio_featureDF.to_gbq(
            destination_table=f'spotify_mpd.track_audio_{split_id}', 
            project_id=f'{project}', # TODO: param
            location='us-central1', 
            # table_schema=schema,
            progress_bar=False, 
            reauth=False, 
            if_exists='append'
            )
      except pandas_gbq.gbq.InvalidSchema as invalid_schema:
        logging.info('invalid schema, skipping')
        pass
      logging.info(f'{count} of {uri_list_length} complete!')
      uri_batch = []
      count += 1
      time.sleep(sleep_param)
    else:
      uri_batch.append(uri)
      count += 1
      
      
  logging.info(f'audio features appended')

  return (
      f'DONE',
  )

### Artists

In [ ]:
### Artist racks api call

@kfp.v2.dsl.component(
    base_image="python:3.9",
    packages_to_install=['fsspec',' google-cloud-bigquery',
                         'google-cloud-storage',
                         'gcsfs',
                         'spotipy','requests','db-dtypes',
                         'numpy','pandas','pyarrow','absl-py', 'pandas-gbq==0.17.4'])
def call_spotify_api_split_artist(
    project: str,
    location: str,
    all_tracks_split_1_gcs_uri: str,
    gcs_bucket: str,
    gcs_prefix_eda: str,
    client_id: str,
    client_secret: str,
    sleep_param: int,
    split_id: int,
) -> NamedTuple('Outputs', [('track_audio_feat_split_1_gcs_uri', str),]):
  print(f'pip install complete')
  import os
  import spotipy
  from spotipy.oauth2 import SpotifyClientCredentials
  import re
  import warnings
  warnings.simplefilter(action='ignore', category=FutureWarning)
  import pandas as pd
  import json
  from io import BytesIO
  import time
  from google.cloud import storage
  import gcsfs
  import numpy as np
  from requests.exceptions import ReadTimeout, HTTPError, ConnectionError, RequestException
  from absl import logging
  from google.cloud import bigquery
  import pandas_gbq
  # print(f'package import complete')

  logging.set_verbosity(logging.INFO)
  logging.info(f'package import complete')

  storage_client = storage.Client(
    project=project
  )
  split='split_1'

  # print(f'Extracting audio features and recommendations for {split} \n')
  logging.info(f'Extracting audio features and recommendations for {split} \n')

  # client_id='2dce494e64a74be980138668f4402b97'
  # client_secret='1706fc14574a4fef9132aaaacb31aa1c'

  logging.info(f'spotipy auth complete')
  def spot_audio_features(uri, client_id, client_secret):
    
    # Authenticate
    client_credentials_manager = SpotifyClientCredentials(
        client_id=client_id, 
        client_secret=client_secret
    )
    sp = spotipy.Spotify(
        client_credentials_manager = client_credentials_manager, 
        requests_timeout=10, 
        retries=10 )
    ############################################################################
    # Create Track Audio Features DF for Split
    ############################################################################

    #Audio features

    #Artist of the track, for genres and popularity
    features = {}
    #tracks API call
    # if tracks:

    #artists api call
    uri = [u.replace('"', '') for u in uri] #fix the quotes 
    artists = sp.artists(uri)

    artist_pop = []
    artist_genres = []
    followers = []
    id_list = uri
    for artist in artists['artists']:
      if artist is not None:
        artist_pop.append(artist['popularity'])
        artist_genres.append(artist['genres'])
        # if artist['followers']['total'] is None:
        followers.append(artist['followers']['total'])
        # else:
        #   followers.append(-1)
      else:
        artist_pop.append(-1)
        artist_genres.append('unknown')

    # logging.info(print(f"artist: {artist_pop}"))
    # logging.info(print(f"genres: {artist_genres}"))
    # logging.info(print(f"followers: {followers}"))
    features["artist_pop"] = artist_pop
    features["genres"] = artist_genres
    features['followers'] = followers
    features['artist_uri'] = id_list
    audio_df = pd.DataFrame(features)
    audio_df['genres'] = audio_df['genres'].astype(str)
    # logging.info(print(audio_df)) #logging
    return audio_df

  bq_client = bigquery.Client(
      project=project, location=location
  )

  

  logging.info(f'finished downloading tracks for split {split_id}')
  query = f"""select count(1) as count from `jtotten-project.spotify_mpd.unique_artists` a where mod(a.duration_ms, 11) = {split_id}-1 
  and a.artist_uri NOT IN (SELECT artist_uri FROM `jtotten-project.spotify_mpd.track_artist_{split_id}`)""" #excluding what we already loaded
  uri_list_length_df = bq_client.query(query).result().to_dataframe()
  n_artists = uri_list_length_df['count'][0]
  logging.info(f'number of distinct artists: {n_artists}')
  audio_featureDF = pd.DataFrame()
  uri_list_length = n_artists
  

  schema = [{'name': 'artist_pop', 'type': 	'INTEGER'	},	
            {'name':'genres', 'type': 	'STRING'	},	
            {'name':'followers', 'type': 	'INTEGER'	},	
            {'name':'artist_uri', 'type': 'STRING'}
  ]
  count = 1
  uri_batch = []
  # handling bad track/artist_uris
  query = f"""select artist_uri from `jtotten-project.spotify_mpd.unique_artists` a where mod(a.duration_ms, 11) = {split_id}-1 
  and a.artist_uri NOT IN (SELECT artist_uri FROM `jtotten-project.spotify_mpd.track_artist_{split_id}`)""" #excluding what we already loaded

  ats = bq_client.query(query).result().to_dataframe()
  artist_set = ats.artist_uri.to_list()
  for uri in artist_set:
    if count % 50 == 0 or uri_list_length == count: #grab a batch of 50 artists
      uri_batch.append(uri)
      ### Try catch block for function
      try:
        audio_featureDF = spot_audio_features(uri_batch, client_id, client_secret)
      except ReadTimeout:
        logging.info("'Spotify timed out... trying again...'")
        audio_featureDF = spot_audio_features(uri_batch, client_id, client_secret)
      except HTTPError as err: #JW ADDED
        logging.info(f"HTTP Error: {err}")
      except spotipy.exceptions.SpotifyException as spotify_error: #jw_added
        logging.info(f"Spotify error: {spotify_error}")
      try:
        audio_featureDF.to_gbq(
            destination_table=f'spotify_mpd.track_artist_{split_id}', 
            project_id=f'{project}', # TODO: param
            location='us-central1', 
            # table_schema=schema,
            progress_bar=False, 
            reauth=False, 
            if_exists='append'
            )
      except pandas_gbq.gbq.InvalidSchema as invalid_schema:
        logging.info('invalid schema, skipping')
        pass
      logging.info(f'{count} of {uri_list_length} complete!')
      uri_batch = []
      count += 1

      time.sleep(sleep_param)

    else:
      uri_batch.append(uri)
      count += 1

  return (
      f'DONE',
  )

## Build Pipeline

In [ ]:
PROJECT_ID = 'jtotten-project'
# PROJECT_ID = 'matching-engine-playlist'
LOCATION = 'us-central1'
BUCKET_NAME = 'matching-engine-content'
# BUCKET_NAME = 'spotify-million-playlists'

PIPELINE_VERSION = 'v3' # pipeline code
PIPELINE_TAG = f'{PIPELINE_VERSION}-spotify-feature-enrich'
print("PIPELINE_TAG:", PIPELINE_TAG)

VERSION = 'v5' # general

PIPELINE_TAG: v3-spotify-feature-enrich


In [ ]:
@kfp.v2.dsl.pipeline(
  name=f'{VERSION}-{PIPELINE_TAG}'.replace('_', '-')
)
def pipeline(
    project: str,
    location: str,
    gcs_bucket: str,
    gcs_prefix_eda: str,
    all_tracks_filename: str,
    bq_project: str,
    bq_dataset: str,
    sleep_param: int = 10,
    parallel_creds: str = json.dumps([
                                      {
                                          'data': 'gs://matching-engine-content/spotify-million-playlist/eda/all_tracks_split_1.csv',
                                          'api_key': '3ab5af00e21b44b0bee976289d9da168',
                                          'secret_': '',
                                          'split': 1
                                      },
                                      {
                                          'data': 'gs://matching-engine-content/spotify-million-playlist/eda/all_tracks_split_2.csv',
                                          'api_key': '52ca2066dfeb434ab832e6e9c709b9e7',
                                          'secret_': '',
                                          'split': 2
                                      },
                                      {
                                          'data': 'gs://matching-engine-content/spotify-million-playlist/eda/all_tracks_split_3.csv',
                                          'api_key': '127935a08a294e5292a79cf54a4fcce2',
                                          'secret_': '',
                                          'split': 3
                                      },
                                      {
                                          'data': 'gs://matching-engine-content/spotify-million-playlist/eda/all_tracks_split_4.csv',
                                          'api_key': 'bc5bde5fae56491c997fb1e351560335',
                                          'secret_': '',
                                          'split': 4
                                      },
                                      {
                                          'data': 'gs://matching-engine-content/spotify-million-playlist/eda/all_tracks_split_5.csv',
                                          'api_key': 'f6f2bc364a7b49bab47c1d6df9f1882a',
                                          'secret_': '',
                                          'split': 5
                                      },
                                      { 
                                          'data': 'gs://matching-engine-content/spotify-million-playlist/eda/all_tracks_split_6.csv',
                                          'api_key': '7283d7c69e2f49dfbe81a07266e512e7',
                                          'secret_': '',
                                          'split': 6
                                      },
                                      { 
                                          'data': 'gs://matching-engine-content/spotify-million-playlist/eda/all_tracks_split_7.csv',
                                          'api_key': '4ff7390e97f643c3a0b4ebc2fa7d9c99',
                                          'secret_': '',
                                          'split': 7
                                      },
                                      { 
                                          'data': 'gs://matching-engine-content/spotify-million-playlist/eda/all_tracks_split_8.csv',
                                          'api_key': '8bf5485e4ee843da80c67da2906b616c',
                                          'secret_': '',
                                          'split': 8
                                      },
                                      { 
                                          'data': 'gs://matching-engine-content/spotify-million-playlist/eda/all_tracks_split_9.csv',
                                          'api_key': '963f4982976443d89b7f052293f6e590',
                                          'secret_': '',
                                          'split': 9
                                      },
                                      { 
                                          'data': 'gs://matching-engine-content/spotify-million-playlist/eda/all_tracks_split_10.csv',
                                          'api_key': '49efce4863e140c4944da92e53ac1200',
                                          'secret_': '',
                                          'split': 10
                                      },
                                      { 
                                          'data': 'gs://matching-engine-content/spotify-million-playlist/eda/all_tracks_split_11.csv',
                                          'api_key': '8cc933ca4892433b92d0478ee4a123e6',
                                          'secret_': '',
                                          'split': 11
                                      }
                                      ],
                                     sort_keys=True)
):

    split_tracks_data_op = split_tracks_data(
      project=project,
      location=location,
      gcs_bucket=gcs_bucket,
      gcs_prefix_eda=gcs_prefix_eda,
      all_tracks_filename=all_tracks_filename,
    )


  with dsl.ParallelFor(parallel_creds) as item:
    call_spotify_api_split_artist_op = call_spotify_api_split_artist(
        project=project,
        location=location,
        all_tracks_split_1_gcs_uri=item.data,
        gcs_bucket=gcs_bucket,
        gcs_prefix_eda=gcs_prefix_eda,
        client_id=item.api_key,
        client_secret=item.secret_,
        sleep_param=3,
        split_id=item.split
    )

    call_spotify_api_split_audio_op = call_spotify_api_split_audio(
        project=project,
        location=location,
        all_tracks_split_1_gcs_uri=item.data,
        gcs_bucket=gcs_bucket,
        gcs_prefix_eda=gcs_prefix_eda,
        client_id=item.api_key,
        client_secret=item.secret_,
        sleep_param=3,
        split_id=item.split
    )

In [ ]:
kfp.v2.compiler.Compiler().compile(
  pipeline_func=pipeline, 
  package_path='custom_container_pipeline_spec.json',
)

/usr/local/lib/python3.7/dist-packages/kfp/v2/compiler/compiler.py:1281: FutureWarning: APIs imported from the v1 namespace (e.g. kfp.dsl, kfp.components, etc) will not be supported by the v2 compiler since v2.0.0
  category=FutureWarning,


In [ ]:
# jtotten-project #
PROJECT_ID = 'jtotten-project'
LOCATION = 'us-central1'
GCS_BUCKET = 'matching-engine-content'
GCS_PREFIX_EDA = 'spotify-million-playlist/eda'
ALL_TRACKS_FILENAME = 'all_tracks_v2.csv'
BQ_PROJECT = 'jtotten-project' 
BQ_DATASET = 'spotify_mpd' # 

In [ ]:
overwrite = True
# overwrite = False

if not PIPELINES.get('train') or overwrite:
  response = pipeline_client.create_run_from_job_spec(
    job_spec_path='custom_container_pipeline_spec.json',
    parameter_values={
        'project': PROJECT_ID,
        'location': LOCATION,
        'gcs_bucket': GCS_BUCKET,
        'gcs_prefix_eda': GCS_PREFIX_EDA,
        'all_tracks_filename': ALL_TRACKS_FILENAME,
        'bq_project': BQ_PROJECT,
        'bq_dataset': BQ_DATASET,
    },
    pipeline_root=f'{GS_PIPELINE_ROOT_PATH}/{VERSION}',
  )
  PIPELINES['train'] = response['name']

# README